In [298]:
import numpy as np
import itertools
import random
import simpy
import csv

In [299]:
# All units are in minutes

WEEKS = 4

# Simulation time in minutes
SIM_TIME = 168 * WEEKS * 60              

# number of docks/berth
DOCKS = 2

# number of tugs 
TUGS = 100

# number of cranes 
CRANES = 4


# shape parameter of Erlang
K = 3

UNLOAD_SPEED = 1
UNLOADING_TIME = 240.0

# average inter-arrival times for ships of type 1 to 3
AVG_TRAVEL_TYPE1_3 = 180.0

# average unloading time for ships of type 1
UNLOAD_TIME_TYPE_1 = UNLOADING_TIME

# average unloading time for ships of type 2
UNLOAD_TIME_TYPE_2 = UNLOADING_TIME

# average unloading time for ships of type 3
UNLOAD_TIME_TYPE_3 = UNLOADING_TIME

BERTHING_TIME = 45.0
DEBERTHING_TIME = 30.0


"""----- Ship distribution by type ----"""
# ship distribution of ships type 1 to 3 {type 1: 25%, type 2: 55%, type 3: 20%}
SHIP_DISTR = [1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3]

"""-----------LOG----------------"""
# wait tug to berthing
tug_wt1 = []
# wait for berth
berth_wt = []
# wait tug to deberthing
tug_wt2 = []
# waiting for crane
crane_wt = []

# help to analyze results
ship_departure_times = []

In [300]:
def ship_generator(env, tug, dock, crane):
    #Generate new ships.
    i = 0
    while True:
        yield env.timeout(1)
        # Randomly select a ship type from SHIP_DISTR
        shiptype = random.choice(SHIP_DISTR)
        env.process(ship('Ship %d' % i, env, tug, dock, crane, shiptype))
        yield env.timeout(time_to_arrive())
        i += 1

def time_to_arrive():
    # Erlang/Gamma distribution with shape k, scale theta
    theta = AVG_TRAVEL_TYPE1_3/K
    return random.gammavariate(K,theta)
    
def time_to_berth():
    return random.expovariate(1.0/BERTHING_TIME)

def time_to_deberth():
    return random.expovariate(1.0/DEBERTHING_TIME)

def time_to_unload(shiptype):
    unloadTime = {
            1 : UNLOAD_TIME_TYPE_1,
            2 : UNLOAD_TIME_TYPE_2,
            3 : UNLOAD_TIME_TYPE_3,
                  }
    return random.expovariate(1.0/unloadTime[shiptype])

def ship(name, env, tug, dock, crane, shiptype):
    print('%s arriving at port at %.1f' % (name, env.now))
    with tug.request() as req:
        # Request the tug
        wt = env.now
        yield req
        wt = env.now - wt
        tug_wt1.append(wt)
        
    # Request for dock and process ship at dock
    env.process(shipAtDock(name, env, tug, dock, crane, shiptype))

        
def tug_transport(name, env, tug, shiptype):
    """Arrives at the port or the dock after a 1h delay and transports the ship."""
    # print('Tug starts %s transport at %.1f' % (name, env.now))
    yield env.timeout(TUG_TIME_ONE_WAY)
    # print('Tug stops %s transport at %.1f' % (name, env.now))


def shipAtDock(name, env, tug, dock, crane, shiptype):
    with dock.request() as req:
        # Request a dock
        wt = env.now
        yield req
        wt = env.now - wt
        berth_wt.append(wt)
        
        # Berthing
        yield env.timeout(time_to_berth())
        
        with crane.request() as req:
            # Request the crane
            wt = env.now
            yield req
            wt = env.now - wt
            crane_wt.append(wt)
            
            # Loading/unloading
            print('%s of shiptype %d unloading at %.1f' % (name, shiptype, env.now))           
            yield env.timeout(time_to_unload(shiptype))
            print('%s unloaded at %.1f' % (name, env.now))
        
        with tug.request() as req:
            # Request the tug
            wt = env.now
            yield req
            wt = env.now - wt
            tug_wt2.append(wt)

        # Deberthing
        yield env.timeout(time_to_deberth())
            
    ship_departure_times.append(env.now)
    print('%s departed at %.1f' % (name, env.now))

In [301]:
# Setup and start the simulation
print('Marine Transport Simulation')

# Create environment and start processes
env = simpy.Environment()

# ships_type_4 = simpy.Resource(env, 1)

# Resource with capacity of usage slots that can be requested by processes.
tug = simpy.Resource(env, TUGS)

# Resource with capacity of usage slots that can be requested by processes.
dock = simpy.Resource(env, DOCKS)

# Resource with capacity of usage slots that can be requested by processes.
crane = simpy.Resource(env, CRANES)

# Process an event yielding generator.
env.process(ship_generator(env, tug, dock, crane))

# Execute!
env.run(until=SIM_TIME)

Marine Transport Simulation
Ship 0 arriving at port at 1.0
Ship 0 of shiptype 2 unloading at 11.5
Ship 1 arriving at port at 91.3
Ship 0 unloaded at 111.1
Ship 0 departed at 148.5
Ship 1 of shiptype 3 unloading at 190.5
Ship 1 unloaded at 266.7
Ship 1 departed at 271.2
Ship 2 arriving at port at 296.2
Ship 2 of shiptype 1 unloading at 303.1
Ship 3 arriving at port at 357.6
Ship 3 of shiptype 1 unloading at 374.3
Ship 4 arriving at port at 580.1
Ship 2 unloaded at 644.3
Ship 2 departed at 646.4
Ship 5 arriving at port at 654.6
Ship 4 of shiptype 1 unloading at 685.7
Ship 3 unloaded at 696.0
Ship 4 unloaded at 697.6
Ship 3 departed at 715.0
Ship 6 arriving at port at 721.5
Ship 5 of shiptype 1 unloading at 742.1
Ship 4 departed at 752.0
Ship 6 of shiptype 1 unloading at 778.0
Ship 5 unloaded at 913.2
Ship 5 departed at 924.8
Ship 7 arriving at port at 996.1
Ship 7 of shiptype 1 unloading at 1030.0
Ship 8 arriving at port at 1129.3
Ship 9 arriving at port at 1221.4
Ship 6 unloaded at 1285

In [302]:
num_ship = min(len(tug_wt1), len(tug_wt2), len(berth_wt),len(crane_wt))
ship_departure_times = ship_departure_times[0:num_ship-1]
ship_delay_times = np.add(np.add(berth_wt[0:num_ship-1], crane_wt[0:num_ship-1]), np.add(tug_wt1[0:num_ship-1], tug_wt2[0:num_ship-1]))

In [303]:
print ship_delay_times

[   0.            0.            0.            0.           66.23109776
   60.39678166   30.5516354     0.          159.24663152  115.43428959
  129.05978254  105.17094498   59.89912008  203.04136555    0.
  147.3612395     0.          110.99525897  112.71741105    0.
    0.            0.            0.          527.71862943  354.27822609
  428.70854766  203.19118715  119.50708448  310.66203808  454.16872434
  457.5133428    60.19249756  345.16777463  176.73444997  177.20845128
   37.96137825  341.7533407   224.01182333  231.39753376    0.
    0.           54.97679129    0.            0.          147.85644554
  221.72980734   33.21954834  236.46734683  310.09592867  400.56954659
  256.33188224  106.97721778  131.68781423    0.            0.
   47.73652719  310.76286249   38.92165596   17.86621925    0.
    0.            0.            0.          177.43268038  103.08097602
  208.47051747    0.            0.           79.93191046  149.67710724
  226.44646362  109.60479063  151.98915275    

In [235]:
# Initialize the file and write first result
with open('simpy_results9.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(["Time", "Delay"])
    for i in range(0,num_ship-1):
        writer.writerow([ship_departure_times[i], ship_delay_times[i]])

In [304]:
# Append to the file if it already exists
with open('simpy_results9.csv', 'a') as f:
    writer = csv.writer(f)
    for i in range(0,num_ship-1):
        writer.writerow([ship_departure_times[i], ship_delay_times[i]])